# Threading (Polyglot)

In [ ]:
#!import ../../lib/fsharp/Notebooks.dib
#!import ../../lib/fsharp/Testing.dib

In [ ]:
#!import ../../lib/fsharp/Common.fs

In [ ]:
open Common

## newDisposableToken

In [ ]:
let inline newDisposableToken (mergeToken: System.Threading.CancellationToken option) =
    let cts = new System.Threading.CancellationTokenSource ()
    let cts =
        match mergeToken with
        | None -> cts
        | Some mergeToken ->
            System.Threading.CancellationTokenSource.CreateLinkedTokenSource [| cts.Token; mergeToken |]
    let disposable = newDisposable cts.Cancel
    cts.Token, disposable

In [ ]:
//// test

let mutable counter = 0

let inline run fn =
    let token, disposable = newDisposableToken None
    use _ = disposable
    fn token
    async {
        fn token
    }
    |> Async.Start

let inline fn (token : System.Threading.CancellationToken) =
    counter <- counter + (if token.IsCancellationRequested then 10 else 1)

async {
    run fn
    do! Async.Sleep 10
    return counter
}
|> Async.RunSynchronously
|> _equal 11

11
